In [ ]:
import requests

def call_qwen_model(prompt: str) -> dict:
    url = "http://localhost:11434/v1"  # Adjust the URL if necessary
    headers = {
        "Authorization": "Bearer xagents-coder",
        "Content-Type": "application/json"
    }
    data = {
        "model": "hosted_vllm/Qwen/Qwen2.5-Coder-7B-Instruct",
        "prompt": prompt,
        "temperature": 0.8,
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()

# Example usage
result = call_qwen_model("Generate SQL query to count distinct vehicle IDs.")
print(result)

In [8]:
from openai import OpenAI
from typing import List, Dict, Union


def math_tool(operation: str, a: float, b: float) -> float:
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    elif operation == "multiply":
        return a * b
    elif operation == "divide":
        return a / b if b != 0 else float("nan")
    else:
        raise ValueError("Invalid operation")


def return_name_tool() -> str:
    return "This is a name tool."


def chat_with_tools(prompt: str, tools: List[Dict]) -> Dict:
    client = OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # Required but unused
    )

    response = client.chat.completions.create(
        model="qwen2.5:7b",  # Updated model name
        messages=[{"role": "user", "content": prompt}],
        tools=tools,
        temperature=0.8,
    )
    return response


# Properly formatted tools with parameters
tools = [
    {
        "type": "function",
        "function": {
            "name": "math_tool",
            "description": "Performs basic math operations.",
            "parameters": {
                "type": "object",
                "properties": {
                    "operation": {
                        "type": "string",
                        "enum": ["add", "subtract", "multiply", "divide"],
                    },
                    "a": {"type": "number"},
                    "b": {"type": "number"},
                },
                "required": ["operation", "a", "b"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "return_name_tool",
            "description": "Returns a name.",
            "parameters": {"type": "object", "properties": {}},
        },
    },
]

# Example usage
prompt = "what is 5/ 3?"
result = chat_with_tools(prompt, tools)
print(result)

ChatCompletion(id='chatcmpl-273', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_a9zmm6fu', function=Function(arguments='{"a":5,"b":3,"operation":"divide"}', name='math_tool'), type='function', index=0)]))], created=1741676453, model='qwen2.5:7b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=32, prompt_tokens=229, total_tokens=261, completion_tokens_details=None, prompt_tokens_details=None))


In [9]:
# After getting the response
message = result.choices[0].message
if message.tool_calls:
    for tool_call in message.tool_calls:
        print(f"Tool call requested: {tool_call.function.name}")
        print(f"Arguments: {tool_call.function.arguments}")

Tool call requested: math_tool
Arguments: {"a":5,"b":3,"operation":"divide"}


In [10]:
message

ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_a9zmm6fu', function=Function(arguments='{"a":5,"b":3,"operation":"divide"}', name='math_tool'), type='function', index=0)])

In [11]:
import json


def process_tool_calls(response):
    client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

    # Get the initial response
    message = response.choices[0].message
    messages = [{"role": "user", "content": prompt}]

    print(message)

    # If there are tool calls, execute them and get results
    if message.tool_calls:
        messages.append(
            {"role": "assistant", "content": None, "tool_calls": message.tool_calls}
        )

        for tool_call in message.tool_calls:
            # Execute the tool
            function_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            try:
                if function_name == "math_tool":
                    result = math_tool(**args)
                elif function_name == "return_name_tool":
                    result = return_name_tool()
                else:
                    result = f"Error: Unknown function {function_name}"
            except Exception as e:
                result = f"Error executing tool: {str(e)}"

            # Add tool response to messages
            messages.append(
                {
                    "role": "tool",
                    "content": str(result),
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                }
            )

        # Get final completion with tool results
        final_response = client.chat.completions.create(
            model="qwen2.5:7b", messages=messages, temperature=0.5
        )
        return final_response
    else:
        return response


# Example usage
# tools = [...]  # The same tool definitions from previous example

prompt = "Use the math_tool to add 5 and 3, and return the result."
initial_response = chat_with_tools(prompt, tools)
final_response = process_tool_calls(initial_response)

# Print the final answer
print(final_response.choices[0].message.content)

The result of adding 5 and 3 is 8.


In [12]:
final_response.choices[0].message

ChatCompletionMessage(content='The result of adding 5 and 3 is 8.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)